In [1]:
import utils.dataset_utils as dataset
import utils.train_utils as train

import numpy as np
import copy
import pickle

from sklearn.base import BaseEstimator, ClassifierMixin
import torch.nn as nn
import torch

from datetime import datetime
from dateutil.relativedelta import relativedelta

In [2]:
def unit_vector_norm(X):
    return (X.T / np.sqrt((X**2).sum(axis=1))).T

In [3]:
file_location = "../data/Raman_Mouse/preprocess/"
filenames = np.load(f"{file_location}FileNames.npy")
with open(f'{file_location}Sample_labels.pickle', 'rb') as f:
    labels = pickle.load(f)

data = []
for f in filenames:
    x = np.load(f"{file_location}{f.split('.')[0]}_raman.npy")
    data.append((unit_vector_norm(x.reshape(-1,x.shape[-1])).reshape(x.shape), labels[f]))


In [4]:
class MLP(nn.Module):
    def __init__(self, size=1300*7*7 , output=10, depth=2, neurons=1300, bias=True, **kwargs):
        super().__init__()

        self.layers = nn.Sequential( 
            nn.Dropout3d(0.25),
            nn.Flatten(),
            nn.Linear(size, neurons, bias=bias),
            nn.ReLU(True),
            *((nn.Linear(neurons, neurons, bias=bias),
            nn.ReLU(True)) * (depth-1)),
            nn.Linear(neurons, output, bias=bias)
        )
  
    def forward(self, x):
        return self.layers(x)

In [5]:
class SupervisedClassifier(BaseEstimator):
    def __init__(self, **kwargs):
        self.kwargs = kwargs

        _use_cuda = torch.cuda.is_available() and kwargs['cuda']
        if _use_cuda:
            torch.backends.cudnn.enabled = True
            torch.backends.cudnn.benchmark = True
        self.device = torch.device('cuda' if _use_cuda else 'cpu')        
        
    def fit(self, data):
        self.model = MLP(**self.kwargs).to(self.device)
        print("here")

        parameters = filter(lambda x: x.requires_grad, self.model.parameters())
        self.optimizer = optim.Adam(parameters)        
        train_loader, test_loader = dataset.load_liver(data, self.kwargs['batch_size'], self.kwargs['patch_size'])
        
#         for epoch in range(self.kwargs['epochs']):
#             print('-'*50)
#             print('Epoch {:3d}/{:3d}'.format(epoch+1, self.kwargs['epochs']))
#             start_time = datetime.now()
#             train.train(self.model, self.optimizer, train_loader, self.kwargs['loss_func'], self.kwargs['log_step'], self.device)
#             end_time = datetime.now()
#             time_diff = relativedelta(end_time, start_time)
#             print('Elapsed time: {}h {}m {}s'.format(time_diff.hours, time_diff.minutes, time_diff.seconds))
#             loss = train.test(self.model, test_loader, self.kwargs['loss_func'], self.device)
#             print('Validation| bits: {:2.2f}'.format(loss), flush=True)    
                         
        # Return the classifier
        return self
    
    def predict(self, X):
        """
        predict transforms the data into the reference space. Min weight should be 0 or higher then 'min_weight'
        The error is the NMSE, where the MSE is normalised by the signal strength. 
        error.shape = X.shape[0], so for each data point the error is calculated.
        """
        # Check is fit had been called
        check_is_fitted(self)

        # Input validation
        X = check_array(X)
#         self.model(X)
        
#         return RCA_vector

In [6]:
BATCH_SIZE = 64
EPOCHS = 10
PATCH_SIZE = 7

kwargs = {'size': PATCH_SIZE*PATCH_SIZE*data[0][0].shape[-1],
          'batch_size': BATCH_SIZE,
          'patch_size': PATCH_SIZE,
          'cuda': False,
          'log_step': 30,
          'epochs': 20,
          'depth': 3,
          'output': 4,
          'loss': nn.CrossEntropyLoss(),
          'neurons': data[0][0].shape[-1]*10,
          'bias': True
         }

In [ ]:
rvc = SupervisedClassifier(**kwargs)
rvc.fit(data)
